In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import pandas_profiling

In [2]:
train_df = pd.read_csv('Dataset/train.csv')
soln_df = pd.read_csv('Dataset/test.csv')

In [3]:
train_df.drop(['pet_id'], axis = 1, inplace = True)
main_test = soln_df.drop(['pet_id'], axis = 1)

In [4]:
def pre_process(df):
    df.condition.fillna(3, inplace=True)
    df['time_to_shelter'] = (pd.to_datetime(df.listing_date) - pd.to_datetime(df.issue_date)).dt.days
    df.drop(['issue_date','listing_date'], axis = 1, inplace=True)
    
    top_colors = [
                     'Black',
                     'White',
                     'Brown',
                     'Brown Tabby',
                     'Tan',
                     'Blue',
                     'Orange Tabby',
                     'Red',
                     'Brown Brindle',
                     'Tricolor',
                     'Blue Tabby',
                     'Tortie',
                     'Calico',
                     'Gray',
                     'Chocolate',
                     'Torbie',
                     'Cream Tabby',
                     'Sable',
                     'Cream',
                     'Fawn',
                     'Yellow',
                     'Buff',
                     'Lynx Point',
                     'Blue Merle'
                ]
    
    def reduce_color(color):
        if color in top_colors:
            return color
        return 'Other'
    
    df['color_mod'] = df.color_type.apply(reduce_color)
    df.drop(['color_type'], axis = 1, inplace=True)
    df.drop(['length(m)','height(cm)'], axis = 1, inplace= True)
    
    return df

In [5]:
train_df = pre_process(train_df)
main_test = pre_process(main_test)

In [6]:
train_y1 = train_df.loc[:,['breed_category']]
train_y2 = train_df.loc[:,['pet_category']]

In [7]:
train_X = train_df.drop(['breed_category','pet_category'], axis=1)

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [9]:
ct = ColumnTransformer([("color", OneHotEncoder(sparse=False, drop='first'), [4])], remainder = 'passthrough')
ct.fit(train_X)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('color',
                                 OneHotEncoder(categories='auto', drop='first',
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=False),
                                 [4])],
                  verbose=False)

In [10]:
train_X = ct.transform(train_X)
main_test = ct.transform(main_test)

In [11]:
print(train_X.shape)
print(main_test.shape)

(18834, 28)
(8072, 28)


In [12]:
from sklearn.model_selection import train_test_split

X_train_breed, X_test_breed, y_train_breed, y_test_breed = train_test_split(train_X, train_y1, test_size = 0.2)
X_train_pet, X_test_pet, y_train_pet, y_test_pet = train_test_split(train_X, train_y2, test_size = 0.2)

In [13]:
from imblearn.over_sampling import RandomOverSampler

os = RandomOverSampler()

X_train_breed, y_train_breed = os.fit_resample(X_train_breed, y_train_breed)
X_train_pet, y_train_pet = os.fit_resample(X_train_pet, y_train_pet)

Using TensorFlow backend.


In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dropout
from sklearn.metrics import confusion_matrix 

In [15]:
X_train_breed.shape

(21702, 28)

In [16]:
y_train_breed.shape

(21702, 1)

In [17]:
def breed_model():

    model = Sequential()
    model.add(Dense(15, input_dim=28, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(15,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(15,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [19]:
def pet_model():

    model = Sequential()
    model.add(Dense(20, input_dim=28, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(20,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(20,  activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(4, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [24]:
estimator = KerasClassifier(build_fn=breed_model, epochs=500, batch_size=50, verbose=1)
estimator.fit(X_train_breed,pd.get_dummies(pd.DataFrame(y_train_breed).astype(str)).values)

Epoch 1/500
21702/21702 [==============================] - 1s 32us/step - loss: 21.9511 - accuracy: 0.3641
Epoch 2/500
21702/21702 [==============================] - 0s 21us/step - loss: 1.6483 - accuracy: 0.3554
Epoch 3/500
21702/21702 [==============================] - 0s 21us/step - loss: 1.2424 - accuracy: 0.4107
Epoch 4/500
21702/21702 [==============================] - 0s 21us/step - loss: 1.1422 - accuracy: 0.4119
Epoch 5/500
21702/21702 [==============================] - 0s 21us/step - loss: 1.1108 - accuracy: 0.4174
Epoch 6/500
21702/21702 [==============================] - 0s 20us/step - loss: 1.0703 - accuracy: 0.4177
Epoch 7/500
21702/21702 [==============================] - 0s 20us/step - loss: 1.0590 - accuracy: 0.4234
Epoch 8/500
21702/21702 [==============================] - 0s 23us/step - loss: 1.0333 - accuracy: 0.4278
Epoch 9/500
21702/21702 [==============================] - 1s 24us/step - loss: 1.0322 - accuracy: 0.4309
Epoch 10/500
21702/21702 [===================

In [25]:
estimator_pet = KerasClassifier(build_fn=pet_model, epochs=500, batch_size=50, verbose=1)
estimator_pet.fit(X_train_pet,pd.get_dummies(pd.DataFrame(y_train_pet).astype(str)).values)

Epoch 1/500
34088/34088 [==============================] - 1s 30us/step - loss: 10.4990 - accuracy: 0.2599
Epoch 2/500
34088/34088 [==============================] - 1s 21us/step - loss: 1.7193 - accuracy: 0.2574
Epoch 3/500
34088/34088 [==============================] - 1s 23us/step - loss: 1.4935 - accuracy: 0.2758
Epoch 4/500
34088/34088 [==============================] - 1s 21us/step - loss: 1.4212 - accuracy: 0.2789
Epoch 5/500
34088/34088 [==============================] - 1s 21us/step - loss: 1.3893 - accuracy: 0.2980
Epoch 6/500
34088/34088 [==============================] - 1s 21us/step - loss: 1.3648 - accuracy: 0.3155
Epoch 7/500
34088/34088 [==============================] - 1s 21us/step - loss: 1.3502 - accuracy: 0.3269
Epoch 8/500
34088/34088 [==============================] - 1s 21us/step - loss: 1.3324 - accuracy: 0.3480
Epoch 9/500
34088/34088 [==============================] - 1s 21us/step - loss: 1.3184 - accuracy: 0.3703
Epoch 10/500
34088/34088 [===================

In [26]:
breed_preds = estimator.predict(X_test_breed)

3767/3767 [==============================] - 0s 18us/step


In [27]:
pet_preds = estimator_pet.predict(X_test_pet)

3767/3767 [==============================] - 0s 18us/step


In [28]:
confusion_matrix(y_test_breed, breed_preds)

array([[1761,    4,    1],
       [ 429, 1282,    0],
       [   0,    0,  290]])

In [29]:
confusion_matrix(y_test_pet, pet_preds)

array([[   5,    0,    1,    3,    0],
       [  16, 1127,  278,   22,    0],
       [  61,   95, 1919,   24,    0],
       [   0,    0,    0,    0,    0],
       [  47,    5,   24,  140,    0]])

In [30]:
breed_preds = estimator.predict(main_test)
pet_preds = estimator_pet.predict(main_test)

8072/8072 [==============================] - 0s 7us/step


In [31]:
submission = pd.DataFrame()

In [32]:
submission['pet_id'] = soln_df.pet_id

In [33]:
submission['breed_category'] = breed_preds

In [34]:
submission['pet_category'] = pet_preds

In [35]:
submission.breed_category = submission.breed_category.astype('int64')

In [36]:
submission

,pet_id,breed_category,pet_category
0,ANSL_75005,1,2
1,ANSL_76663,0,1
2,ANSL_58259,0,2
3,ANSL_67171,0,2
4,ANSL_72871,0,2
...,...,...,...
8067,ANSL_66809,0,3
8068,ANSL_59041,1,2
8069,ANSL_60034,1,2
8070,ANSL_58066,2,0


In [38]:
submission.to_csv('submission_6.csv',index=False)